In [96]:
# importing package
import pandas as pd
import numpy as np
import bs4
import requests
import biothings.utils.dataload as bioloading
import re
import os

In [97]:
# predefined variable
TTD_WEB_LINK = "https://db.idrblab.net/ttd/full-data-download"

# Predefined function

In [98]:
def loadingPage(url):
    request = requests.get(url)
    return bs4.BeautifulSoup(request.text)

def obtain_txt_file(txt_url):
    # creating a new output directory if needed
    folder_path = os.getcwd() + '/output_raw_txtFile'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # write down the txt file
    output_fileName = txt_url[txt_url.rfind('/')+1:txt_url.rfind('.')]
    with open(f'output_raw_txtFile/f{output_fileName}.txt', 'w') as file:
        file.write(requests.get(txt_url).text)

# Actual Program

In [100]:
# loading the whole Web Page
main_webPage = loadingPage(TTD_WEB_LINK)

# collecting link to each individual txt file
doc_collections = main_webPage.findAll('td', attrs={'class' : "v-center"})


# downloading file
print('start downloading file...')
nontxt_files = []
txt_files = []
for doc in doc_collections:
    txt_url = 'https://db.idrblab.net/' + doc.find('a').get('href')

    if txt_url[txt_url.rfind('.')+1:] == 'txt':
        obtain_txt_file(txt_url)
        txt_files.append(txt_url)
    else:
        nontxt_files.append(txt_url)
print(f'processed {txt_files}')
print(f"can't process {nontxt_files}")

start downloading file...
processed ['https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-01-TTD_target_download.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-02-TTD_drug_download.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-03-TTD_crossmatching.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-04-Drug_synonyms.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-05-Drug_disease.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-06-Target_disease.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-08-Biomarker_disease.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P1-09-Target_compound_activity.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P2-01-TTD_uniprot_all.txt', 'https://db.idrblab.net//ttd/sites/default/files/ttd_database/P2-02-TTD_uniprot_successful.txt', 'https://db.idrblab.net//ttd/sites/default/

In [106]:
play = list(bioloading.tabfile_feeder('output_raw_txtFile/fP1-01-TTD_target_download.txt'))

big_list = []
coll = []
for i in play:
    if len(i) == 0:
        big_list.append(coll)
        coll = []
    else:
        coll.append(i)

In [114]:
print(big_list[0])
print(big_list[1])
print(big_list[2])

[['TTD - Therapeutic Targets Database Full Data Download File'], ['Title - TTD targets information in raw format'], ['Version 10.1.01 (2024.01.10)'], ['Provided by IDRB   Lab of Innovative Drug Reasearch and Bioinformatics'], ['                   College of Pharmaceutical Sciences'], ['                   Zhejiang University'], ['                   https://idrblab.org/'], ['Provided by BIDD   BioInformatic and Drug Design Group'], ['                   Department of Pharmacy'], ['                   National Unviersity of Singapore'], ['                   https://bidd.group/'], ['Any question about data provided here, please contact with:'], ['Dr. Zhou (zhou_ying@zju.edu.cn) and Dr. Zhang (zhangyintao@zju.edu.cn)']]
[['--------------------------------------------------------------------------------------------------------'], ['Abbreviations:'], ['TARGETID', 'TTD Target ID'], ['FORMERID', 'TTD Former Target ID'], ['UNIPROID', 'Uniprot ID'], ['TARGNAME', 'Target Name'], ['GENENAME', 'Target